## 10.1 분산을 기준으로 수치 특성 선택하기
수치형 특성 중에서 분산이 낮은 특성(즉 정보가 거의 없는 특성)을 삭제하고 싶습니다.

주어진 기준값보다 높은 분산을 가진 특성을 선택합니다.

In [3]:
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

iris = datasets.load_iris()

features = iris.data
target = iris.target

# 기준값을 만듭니다.
thresholder = VarianceThreshold(threshold=.5)

# 기준값보다 높은 특성을 선택합니다.
features_high_variance = thresholder.fit_transform(features)

# 선택한 특성을 확인합니다.
features_high_variance[0:3]

array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2]])

분산 기준 설정(variance thresholding, VT)는 가장 기본적인 특성 선택 방법 중 하나입니다. 이 방식은 분산이 높은 특성보다 분산이 낮은 특성이 효과적이거나 유용하지 않다는 아이디어에 기반합니다. VT는 먼저 각 특성의 분산을 계산합니다.

$$Var(x) = \frac{1}{n}\sum_{i=1}^n(x_i-\mu)^2$$

여기에서 $x$는 특성 벡터이고 $x_i$는 개별 특성값입니다. $\mu$는 특성의 평균값입니다. 그다음 분산이 기준값에 미치지 못하는 모든 특성을 삭제합니다.

VT를 사용할 때 두 가지를 기억해야 합니다. 첫째, 분산은 원점에 맞춰진 값이 아닙니다. 즉 특성의 제곱 단위입니다. 따라서 특성의 단위가 서로 다르면 VT가 작동하지 않습니다(예를 들어 한 특성은 년 단위이고 다른 특성은 원 단위인 경우). 둘째, 분산의 기준값을 수동으로 선택하기 때문에 어떤 값이 좋은지 판단할 수 있어야 합니다(또는 12장에서 소개하는 모델 선택 기법을 사용합니다). variances_ 속성에서 각 특성의 분산을 확인할 수 있습니다.

In [4]:
# 분산을 확인합니다.
thresholder.variances_

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

마지막으로 특성이 (평균이 0이고 단위 분산으로) 표준화되어 있으면 당연하지만 분산 기준 선택 방식은 올바르게 작동하지 않습니다.

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features_std = scaler.fit_transform(features)

selector = VarianceThreshold()
selector.fit(features_std).variances_

array([1., 1., 1., 1.])

## 10.2 분산을 기준으로 이진 특성 선택하기
이진 범주형 특성(binary categorical feature)에서 분산이 낮은 특성(즉 적은 정보를 가진 특성)을 삭제하고 싶습니다.

Bernoulli random variable의 분산이 기준값 이상인 특성을 선택합니다.

In [6]:
from sklearn.feature_selection import VarianceThreshold

# 예제 특성 행렬을 만듭니다.
# 특성 0: 80%가 클래스 0
# 특성 1: 80%가 클래스 1
# 특성 2: 60%가 클래스 0, 40%는 클래스 1
features = [[0, 1, 0],
            [0, 1, 1],
            [0, 1, 0],
            [0, 1, 1],
            [1, 0, 0]]

# 분산을 기준으로 선택합니다.
thresholder = VarianceThreshold(threshold=(.75 * (1-.75)))
thresholder.fit_transform(features)

array([[0],
       [1],
       [0],
       [1],
       [0]])

수치형 특성과 마찬가질 정보가 많은 범주형 특성을 선택하는 한 가지 전략은 분산을 조사하는 것입니다. 이진 특성(즉 베르누이 확률 변수)의 분산은 다음과 같이 계산합니다.
$$Var(x) = p(1-p)$$

여기에서 $p$는 클래스 1의 샘플 비율입니다. 따라서 $p$값을 설정하여 샘플의 대다수가 한 개의 클래스에 속한 특성을 삭제할 수 있습니다.

이진 특성의 경우에도 variances_ 속성에서 분산을 확인할 수 있습니다.

In [7]:
thresholder.variances_

array([0.16, 0.16, 0.24])

VarianceThreshold 클래스는 수치 특성, 이진 특성에 상관없이 넘파이 var 함수를 사용하여 분산을 계산합니다. 다음 코드는 넘파이 var 함수를 사용하여 앞서 구한 분산을 동일하게 계산합니다.

In [8]:
import numpy as np
np.var(features, axis=0)

array([0.16, 0.16, 0.24])

이진 특성에 var 함수를 사용하는 것은 이진 특성일 때 베르누이 확률 변수의 분산과 같기 때문입니다. 분산 공식을 사용해 간단히 유도해볼 수 잇습니다. 먼저 분산 식을 다음과 같이 풀어 쓸 수 있습니다.
$$Var(x) = \frac{1}{n}\sum_{i=1}^n(x_i-\mu)^2 = \frac{1}{n}(\sum_{i=1}^n x_{i}^2-2\mu\sum_{i=1}^n x_{i}+n\mu^2)$$

0, 1로 이루어진 이진 특성일 경우 $x_{i}^2$은 $x_{i}$와 같으므로 $\frac{1}{n}$을 곱하면 첫 번째 항은 평균과 같아집니다. 두 번째 항도 마찬가지로 $\frac{1}{n}$을 곱하면 평균의 제곱으로 표현할 수 있습니다. 결국 다음과 같이 정리됩니다.
$$= \frac{1}{n}\sum_{i=1}^n x_{i} -2\mu\frac{1}{n}\sum_{i=1}^n x_{i} + \mu^2 = \mu - 2\mu^2 + \mu^2 = \mu - \mu^2 = \mu(1-\mu)$$

이진 특성의 평균 $\mu$는 클래스 1의 샘플 비율과 같습니다. 따라서 var 함수로 이진 특성의 분산을 계산하면 베르누이 확률 변수의 분산 $p(1-p)$와 같습니다. 

threshold 매개변수의 기본값은 0으로 모든 특성을 선택합니다.

## 10.3 상관관계가 큰 특성 다루기
특성 행렬에서 일부 특성의 상관관계가 크다고 의심됩니다.

상관관계 행렬(correlation matrix)을 사용하여 상관관계가 큰 특성을 확인하고 이들 중 하나를 삭제합니다.

In [2]:
import pandas as pd
import numpy as np

# 상관관계가 큰 투 개의 특성을 가진 특성 행렬을 만듭니다.
features = np.array([[1, 1, 1],
                     [2, 2, 0],
                     [3, 3, 1],
                     [4, 4, 0],
                     [5, 5, 1],
                     [6, 6, 0],
                     [7, 7, 1],
                     [8, 7, 0],
                     [9, 7, 1]])

# 특성 행렬을 데이터프레임으로 변환합니다.
dataframe = pd.DataFrame(features)

# 상관관계 행렬을 만듭니다.
corr_matrix = dataframe.corr().abs()

# 상관관계 행렬의 상삼각(upper triangle) 행렬을 만듭니다.
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# 상관 계수가 0.95보다 큰 특성 열의 인덱스를 찾습니다.
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# 특성을 삭제합니다.
dataframe.drop(dataframe.columns[to_drop], axis=1).head(3)

,0,2
0,1,1
1,2,0
2,3,1


머신러닝에서 흔히 부딪히는 한 가지 문제는 상간관계가 큰 특성입니다. 두 가지 특성의 상관관계가 크다면, 담고 있는 정보가 매우 비슷하므로 중복된 특성을 포함하는 것과 같습니다. 이런 특성을 다루는 해결은 간단합니다. 특성 중 하나를 특성 행렬에서 삭제하면 됩니다.

해결에서 첫째, 모든 특성에 대한 상관관계 행렬을 만들었습니다.

In [3]:
# 상관관계 행렬
dataframe.corr()

,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,-0.034503
2,0.000000,-0.034503,1.000000


둘째, 상관관계 행렬의 상삼각 행렬(upper triangle matrix)을 살펴서 크게 상관된 특성의 쌍을 확인합니다.

In [4]:
# 상관관계 행렬의 상삼각 행렬
upper

,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,0.034503
2,NaN,NaN,NaN


셋째, 특성 행렬에서 이런 특성 중 하나를 삭제합니다.

상관관계 행렬은 넘파이 corrcoef 함수로 구할 수도 있습니다. 이 함수는 특성이 행에 놓여 있을 것으로 기대합니다. 특성이 열에 놓여 있다고 알려주려면 rowvar 매개변수를 False로 지정합니다.

In [5]:
np.corrcoef(features, rowvar=False)

array([[ 1.        ,  0.97610336,  0.        ],
       [ 0.97610336,  1.        , -0.03450328],
       [ 0.        , -0.03450328,  1.        ]])

해결에 나온 np.triu 함수는 주어진 배열에서 상삼각 행렬을 추출하여 반환합니다. 매개변수 k가 기본값 0이면 반환되는 행렬에 대각원소가 포함됩니다. k 값이 커질수록 대각원소에서 k만큼 떨어진 삼각행렬을 반환합니다. 예를 들어 k=2일 경우 주대각선에서 2만큼 떨어진 원소부터 포함됩니다.

In [13]:
np.triu(np.ones((4, 4)), k=2)

array([[0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

이와 비슷하게 하삼각 행렬을 구하는 np.tril 함수도 있습니다. 사용법은 triu()와 동일합니다.

In [11]:
np.tril(np.ones((4, 4)), k=0)

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

## 10.4 분류 작업에 관련 없는 특성 삭제하기
범주형 타깃 벡터에서 관련 없는 특성을 삭제하고 싶습니다.

범주형 특성이라면 각 특성과 타깃 벡터 사이의 카이제곱($\chi^2$) 통계를 계산합니다.

In [15]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

# 데이터를 로드합니다.
iris = load_iris()
features = iris.data
target = iris.target

# 범주형 데이터를 정수형으로 변환합니다.
features = features.astype(int)

# 카이제곱 통곗값이 가장 큰 특성 두 개를 선택합니다.
chi2_selector = SelectKBest(chi2, k=2)
features_kbest = chi2_selector.fit_transform(features, target)

# 결과를 확인합니다.
print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


특성이 수치형이면 각 특성과 타깃 벡터 사이에서 분산 분석(ANOVA)의 F-값을 계산합니다.

In [16]:
# F-값이 가장 높은 특성 두 개를 선택합니다.
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

# 결과를 확인합니다.
print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


특정 특성 개수를 선택하는 대신 SelectPercentile를 사용하여 특성의 상위 $n$ 퍼센트를 선택할 수 있습니다.

In [17]:
from sklearn.feature_selection import SelectPercentile

# 가장 큰 F-값의 상위 75% 특성을 선택합니다.
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)

# 결과를 확인합니다.
print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 3


카이제곱 통계는 두 범주형 벡터의 독립성을 평가합니다. 즉, 이 통계는 범주형 특성의 각 클래스별 샘플 빈도와 이 특성이 타깃 벡터와 독립적이라면 (즉 관계가 없다면) 기대할 수 있는 값 사이의 차이입니다.
$$\chi^2 = \sum^n_{i=1}\frac{(O_i - E_i)^2}{E_i}$$

여기에서 $O_i$는 클래스 $i$의 샘플 빈도입니다. $E_i$는 특성과 타깃 벡터 사이에 관계가 없을 때 기대할 수 있는 클래스 $i$의 샘플 빈도입니다.

카이제곱 특성은 관찰 빈도 $O_i$와 전혀 관계가 없다고 기대하는 빈도 $E_i$ 사이에 얼마나 큰 차이가 있는지 알려주는 하나의 숫자입니다. 특성과 타깃 벡터 사이의 카이제곱 통계를 계산하면 둘 사이의 독립성을 측정할 수 있습니다. 특성 변수가 타깃에 독립적이면 분류 문제에 사용할 정보가 없기 때문에 목적에 맞지 않습니다. 다른 한편으로 두 변수가 크게 의존적이면 모델 훈련에 필요한 정보가 많을 것입니다.

특성 선택에서 카이제곱을 사용하려면 각 성과 타깃 벡터 사이의 카이제곱 통계를 계산하면 카이제곱 통계가 가장 좋은 특성을 선택해야 합니다. 사이킷런에서는 SelectKBset를 사용하여 통곗값이 가장 좋은 특성을 선택할 수 있습니다. 매개변수  k는 선택하려는 특성의 개수를 결정합니다. 카이제곱 통계는 두 범주형 벡터 사이에서만 계산할 수 있다는 점을 유념하세요. 이런 이유 때문에 특성 선택으로 카이제곱을 사용하려면 타깃 벡터와 특성이 범주형이어야 합니다. 수치형 특성이 있다면 수치형을 번주형 특성으로 변환하여 카이제곱 특성을 사용할 수 있습니다. 마지막으로 카이제곱 방식을 사용하려면 모든 값이 음수가 아니어야 합니다. 

또는 수치형 특성이라면 f_classif 사용하여 각 특성과 타깃 벡터 사이에 분산 분석(ANOVA)의 F-값 통계를 계산할 수 있습니다. F-값 점수는 타깃 벡터로 수치형 특성을 그룹핑하여 각 그룹의 평균이 크게 차이나는지 평가합니다. 예를 들어 이진 타깃 벡터인 성별과 수치형 특성인 시험 점수가 있다면, F-값 점수는 남성의 평균 테스트 점수가 여성의 평균 테스트 점수와 다른지를 설명합니다. 그렇지 않다면 시험 점수는 성별을 예측하는 데 도움이 되지 않기 때문에 이 특성은 관련성이 없습니다.

load_iris 함수에서 제공하는 붓꽃 데이터셋은 수치형 특성입니다. 해결에서는 카이제곱의 예를 위해 강제로 정수 타입으로 바꾸어 범주형처럼 다루었습니다. 이 데이터를 사용하여 카이제곱 통계를 직접 계산해보죠.

붓꽃 데이터셋은 순서대로 세 개의 꽃 종류(클래스)가 50개식 150개의 샘플로 이루어져 있습니다. 타깃 데이터를 출력해보면 세 클래스가 순서대로 50개씩 놓인 것을 확인할 수 있습니다.

In [18]:
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

관찰 빈도를 구하려면 클래스별로 특성값을 더해야 합니다. 데이터가 클래스 순서대로 50개씩 나열되어 있으므로 특성 행렬의 차원을 (150, 4)에서 (3, 50, 4)로 바꾸어 클래스별 합을 간단히 구할 수 있습니다.

In [19]:
observed = np.sum(features.reshape(3, 50, 4), axis=1)
observed

array([[230, 152,  50,   0],
       [274, 116, 191,  50],
       [304, 129, 255,  79]])

특성이 타깃과 전혀 관계없다면 기대 빈도는 전체 합을 클래스 개수 3으로 나눈 값이 됩니다.

In [20]:
expected = features.sum(axis=0) / 3
expected

array([269.33333333, 132.33333333, 165.33333333,  43.        ])

이제 카이제곱 공식에 위에서 구한 observed와 expected를 대입합니다.

In [21]:
np.sum((observed - expected)**2 / expected, axis=0)

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

카이제곱 값이 큰 세 번째, 네 번째 특성이 선택됩니다. 이 카이제곱 점수는 chi2_selector 객체의 scores_ 속성에 저장되어 있습니다.

In [22]:
chi2_selector.scores_

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

F-값의 공식은 다음과 같습니다.
$$F = \frac{SS_{between}/(k-1)}{(SS_{tot}-SS_{between})/(n-k)}$$

여기에서 $k$는 클래스 개수이고 $n$은 샘플 개수입니다. $SS_{between}$과 $SS_{tot}$는 각각 다음과 같습니다.
$$SS_{between} = \sum^k_{j=1}n_j(\bar{x_j}-\bar{x})^2,  SS_{tot}=\sum^n_{i=1}n_j(x_i-\bar{x})^2$$

$\bar{x}$는 전체 평균이고 $\bar{x_j}$는 클래스별 평균을 나타냅니다. 먼저 넘파이 mean 함수를 사용해 전체 평균과 클래스 평균을 계산해보죠.

In [23]:
total_mean = np.mean(features, axis=0)
total_mean

array([5.38666667, 2.64666667, 3.30666667, 0.86      ])

In [24]:
class_mean = np.mean(features.reshape(3, 50, 4), axis=1)
class_mean

array([[4.6 , 3.04, 1.  , 0.  ],
       [5.48, 2.32, 3.82, 1.  ],
       [6.08, 2.58, 5.1 , 1.58]])

클래스 평균은 앞에서와 마찬가지로 특성 행렬을 (3, 50, 4) 크기로 바꾸어 계산했습니다. 전체 평균과 클래스별 평균을 구하고 나면 나머지는 간단합니다. ss_between 값부터 계산해보죠.

In [25]:
ss_between = np.sum(50 * (class_mean - total_mean)**2, axis=0)
ss_between

array([ 55.41333333,  13.29333333, 440.01333333,  63.88      ])

붓꽃 데이터셋은 클래스별로 50개의 샘플이 있다는 것을 알고 있으므로 따로 개수를 세지 않았습니다. 다음은 ss_total을 계산합니다.

In [26]:
ss_total = np.sum((features - total_mean)**2, axis=0)
ss_total

array([105.57333333,  42.27333333, 467.89333333,  76.06      ])

계산된 ss_between과 ss_total을 F-값 공식에 대입합니다.

In [27]:
f = (ss_between/(3-1)) / ((ss_total - ss_between)/(150-3))
f

array([  81.19776715,   33.71497585, 1160.00645624,  385.48275862])

F-값도 scores_ 속성에서 확인할 수 있습니다.

In [29]:
fvalue_selector.scores_

array([  81.19715 ,   33.715004, 1160.0116  ,  385.483   ], dtype=float32)

ANOVA는 각 특성이 독립적으로 평가되기 때문에 일변량 분석이라고도 부릅니다. 회귀일 때는 f_classif 대신 f_regression 함수를 사용합니다.

## 10.5 재귀적 특성 제거하기
자동으로 최선의 특성을 선택하고 싶습니다.

사이킷런의 RFECV를 사용하여 재귀적 특성 제거(recursive feature elimination, RFE)를 교차검증(cross-validation, CV)으로 수행할 수 있습니다. 모델 성능(예를 들면 정확도)이 나빠질 때까지 특성을 제거하면서 반복적으로 모델을 훈련합니다. 결국 최선의 특성이 남습니다.

In [2]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model

# 특성 행렬과 타깃 벡터를 생성합니다.
features, target = make_regression(n_samples = 1000,
                                   n_features = 100,
                                   n_informative = 2,
                                   random_state = 1)

# 선형 회귀 모델을 만듭니다.
ols = linear_model.LinearRegression()

# 재귀적으로 특성을 제거합니다.
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target)
rfecv.transform(features)

array([[ 0.14925054,  0.94400923],
       [-0.50702314, -1.08511556],
       [-1.74911237,  0.83782064],
       ...,
       [-0.48969413, -0.00832094],
       [-1.06119079, -1.07171859],
       [ 0.14314375,  0.66937732]])

RFE를 수행하고 난 후 남은 특성 개수를 확인합니다.

In [3]:
# 최선의 특성 개수
rfecv.n_features_

2

어떤 특성이 남았는지 확인합니다.

In [4]:
# 선택된 특성이 표시된 불리언 마스크
rfecv.support_

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

특성의 순위를 확인할 수도 있습니다.

In [5]:
# 특성의 순위: 최고(1)에서 최악(96)까지
rfecv.ranking_

array([55, 76, 62, 78, 99, 45, 21, 72, 11, 69, 68, 71, 70, 52, 64, 97, 53,
       48, 27,  8, 24, 13, 29, 51,  7, 88, 49, 56, 15, 47, 38, 26,  1, 31,
       85, 14, 65, 98, 19, 33, 34, 84, 36, 86, 22, 57,  5, 91, 63, 83, 16,
       46, 87, 41, 42, 37, 92, 28, 60, 66, 35, 44, 94, 67, 75, 17, 93, 96,
       89, 81, 40, 32, 95, 30,  4,  3, 43, 90, 82, 59, 61, 18,  6, 39, 50,
        1, 74, 23, 77, 25, 79, 80, 20, 12, 10, 58,  2, 73,  9, 54])

이 레시피는 지금까지 이 책에서 나온 것 중 가장 고급 기술입니다. RFE 이면의 아이디어는 linear regression이나 support vector machine과 같이 parameter(weight 또는 coefficient라고도 부릅니다)를 가진 모델을 반복적으로 훈련하는 것입니다. 맨 처음 모델을 훈련할 때 모든 특성이 포함됩니다. 그다음 가장 작은 파라미터를 가진 특성을 찾습니다(특성의 스케일이 조정되거나 표준화되었다고 가정합니다). 이런 특성은 덜 중요하다는 것을 의미하므로 사용했던 특성 중에서 제거합니다.

자연스럽게 얼마나 많은 특성을 남겨야 하는지가 궁금해집니다. (어떤 가설을 세운 다음) 하나의 특성이 남을 때까지 이 과정을 반복할 수 있습니다. 더 좋은 방법은 교차검증이라는 새로운 개념을 적용하는 것입니다. 

CV를 사용하여 RFE 과정에서 남길 특성의 최적 개수를 찾을 수 있습니다. 구체적으로 매 반복 후에 CV를 사용한 RFE에서는 교차검증을 사용하여 모델을 평가합니다. 특성을 제거한 후에 모델의 CV 결과가 향상되었다면 다음 반복으로 계속 진행합니다. 그러나 어떤 특성을 제거한 후에 모델의 CV 결과가 더 나빠지면 삭제한 특성을 다시 복원하고 이 특성 조합을 최선으로 선택합니다.

사이킷런에서 CV를 사용한 RFE는 RFECV에 구현되어 있습니다. 여기에는 여러 가지 중요한 매개변수가 있습니다. estimator 매개변수에는 훈련할 모델의 객체를 전달합니다(예를 들면 선형 회귀). step 매개변수는 매 반복에서 삭제할 특성의 개수나 비율을 정합니다. scoring 매개변수에는 교차검증 동안 사용할 모델의 평가 지표를 설정합니다.

step 매개변수의 기본값은 1입니다. scoring 매개변수를 지정하지 않으면 estimator에 지정된 모델의 score 메서드를 사용합니다. 기본적으로 회귀일 때는 $R^2$ 점수, 분류일 때는 정확도입니다.

n_jobs 매개변수에서 교차검증을 위해 사용할 CPU 코어 수를 지정할 수 있습니다. n_jobs 매개변수의 기본값은 1입니다. cv 매개변수는 k-fold 교차검증의 k값을 결정합니다. 기본값은 3이고 사이킷런 0.22 버전부터 5로 바뀔 예정입니다. 

사이킷런은 교차검증을 사용하지 않는 재귀적 특성 제거 방법인 RFE 클래스도 제공합니다. RFECV 클래스와 마찬가지로 남길 최소 특성의 개수를 n_features_to_select 매개변수에서 지정할 수 있지만 RFECV와 달리 입력 특성의 절반이 기본값입니다. 해결에서와 동일하게 2개의 특성이 남도록 RFE 객체를 훈련시켜보겠습니다.

In [7]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=ols, n_features_to_select=2)
rfe.fit(features, target)
rfe.transform(features)

array([[ 0.14925054,  0.94400923],
       [-0.50702314, -1.08511556],
       [-1.74911237,  0.83782064],
       ...,
       [-0.48969413, -0.00832094],
       [-1.06119079, -1.07171859],
       [ 0.14314375,  0.66937732]])

rfe 객체가 선택한 특성이 rfecv 객체가 선택한 특성과 동일한지 확인하기 위해 불리언 마스크를 비교해봅니다. 넘파이 all 함수는 모든 원소가 True인지를 검사합니다.

In [9]:
import numpy as np

np.all(rfe.support_ == rfecv.support_)

True